In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import datetime as dt
path = './Fitabase Data 4.12.16-5.12.16'

filenames =  os.listdir(path)
print(filenames)

for filename in filenames:
    globals()[filename.split('.')[0]] = pd.read_csv(f"{path}/{filename}") 
    print(filename.split('.')[0])
    #globals() function allows us to name the dataframes with the filenames. 
    #I used split() to not include .csv part in the filename

['dailySteps_merged.csv', 'heartrate_seconds_merged.csv', 'minuteSleep_merged.csv', 'hourlyCalories_merged.csv', 'dailyCalories_merged.csv', 'hourlySteps_merged.csv', 'minuteIntensitiesWide_merged.csv', 'minuteMETsNarrow_merged.csv', '.git', 'sleepDay_merged.csv', 'main2.ipynb', 'weightLogInfo_merged.csv', 'dailyActivity_merged.csv', 'dailyIntensities_merged.csv', '.venv', 'hourlyIntensities_merged.csv', 'minuteStepsNarrow_merged.csv', 'minuteCaloriesWide_merged.csv', 'minuteIntensitiesNarrow_merged.csv', 'minuteStepsWide_merged.csv', 'minuteCaloriesNarrow_merged.csv', 'main.ipynb']
dailySteps_merged
heartrate_seconds_merged
minuteSleep_merged
hourlyCalories_merged
dailyCalories_merged
hourlySteps_merged
minuteIntensitiesWide_merged
minuteMETsNarrow_merged


IsADirectoryError: [Errno 21] Is a directory: './/.git'

We will only be using daily, hourly, and sleep datasets in this analysis. 

<a id="cleaning-dataset"></a>  
## Cleaning and Processing

##### Let's take a look at the datasets to get a better idea.

In [ ]:
print(len(hourlySteps_merged))
print(len(hourlyCalories_merged))
print(len(dailyCalories_merged))
print(len(dailySteps_merged))
print(len(sleepDay_merged))
print(len(heartrate_seconds_merged))

22099
22099
940
940


NameError: name 'sleepDay_merged' is not defined

I already checked out other files and there are 940 rows in daily dataframes (sleepDay_merged has 413) and 22099 rows in hourly dataframes. Heartrate dataframe has 2483658 rows and we will use that later. So we can merge hourly dataframes together to create one dataframe and daily dataframes together to create a daily dataframe. But, I will create a special dataframe for daily analysis with sleep data, it will have less rows than the normal daily dataframe. 

In [ ]:
#Let's checkout the dataframes
hourlySteps_merged.head(3),hourlyCalories_merged.head(3),dailySteps_merged.head(3), dailyActivity_merged.head(3)

Hourly dataframes are pretty consistant and we can merge them by Id and ActivityHour. But there is inconsistency in the naming scheme of daily activity and other daily dataframes (ActivityDay = ActivityDate). So, we will get some redundent rows while merging. But we can remove them later so no worries.

#### Merging datasets to create hourly and daily datasets for Easy Analysis.   
  <a id="daily"></a>  
  ##### Daily data

In [ ]:
#Merging Daily Data
dailyStuff = pd.merge(dailyActivity_merged, dailyCalories_merged, left_on = ["Id","ActivityDate"],
                  right_on = ["Id","ActivityDay"])
dailyStuff = pd.merge(dailyStuff,dailyIntensities_merged, on = ["Id","ActivityDay"])
dailyStuff = pd.merge(dailyStuff,dailySteps_merged, on = ["Id","ActivityDay"])

#Making Activity Day datetime and getting only the date part to merge with sleep data (now this column is not datetime)
dailyStuff['ActivityDay'] = pd.to_datetime(dailyStuff['ActivityDay']).dt.date

#making sleepday's SleepDay column datetime and getting only the date
#Since, everydays sleep time is 12.00 AM, We dont have much use of that data (now this column is not datetime)
sleepDay_merged['SleepDay']= pd.to_datetime(sleepDay_merged['SleepDay']).dt.date

#creating a new dataframe  with sleepdata - this dataframe have less rows than dailyStuff dataframe
dailyStuff_with_Sleep = pd.merge(dailyStuff,sleepDay_merged, how = 'right',left_on = ["Id","ActivityDay"],
                                  right_on = ["Id","SleepDay"])

dailyStuff.head(4),dailyStuff_with_Sleep.head(4)

In [ ]:
#Let's remove these columns because they are redundant and we don't need em.
dailyStuff.drop(['ActivityDate','Calories_y','SedentaryMinutes_y','LightlyActiveMinutes_y','SedentaryActiveDistance_y',
                 'FairlyActiveMinutes_y','VeryActiveMinutes_y','LightActiveDistance_y','ModeratelyActiveDistance_y',
                'VeryActiveDistance_y','StepTotal'],inplace = True,axis = 1)
dailyStuff_with_Sleep.drop(['ActivityDate','SleepDay','Calories_y','SedentaryMinutes_y','LightlyActiveMinutes_y',
                            'SedentaryActiveDistance_y','FairlyActiveMinutes_y','VeryActiveMinutes_y',
                            'LightActiveDistance_y','ModeratelyActiveDistance_y', 'VeryActiveDistance_y',
                            'StepTotal'],inplace = True,axis = 1)

#and now lets rename some columns
dailyStuff.rename(columns = {'Calories_x':'Calories','SedentaryMinutes_x':'SedentaryMinutes',
                            'LightlyActiveMinutes_x':'LightlyActiveMinutes','FairlyActiveMinutes_x':'FairlyActiveMinutes',
                            'VeryActiveMinutes_x':'VeryActiveMinutes','SedentaryMinutes_x':'SedentaryMinutes',
                            'LightActiveDistance_x':'LightActiveDistance','ModeratelyActiveDistance_x':'ModeratelyActiveDistance',
                            'VeryActiveDistance_x':'VeryActiveDistance','SedentaryActiveDistance_x':'SedentaryActiveDistance'},inplace = True)
dailyStuff_with_Sleep.rename(columns = {'Calories_x':'Calories','SedentaryMinutes_x':'SedentaryMinutes',
                            'LightlyActiveMinutes_x':'LightlyActiveMinutes','FairlyActiveMinutes_x':'FairlyActiveMinutes',
                            'VeryActiveMinutes_x':'VeryActiveMinutes','SedentaryMinutes_x':'SedentaryMinutes',
                            'LightActiveDistance_x':'LightActiveDistance','ModeratelyActiveDistance_x':'ModeratelyActiveDistance',
                            'VeryActiveDistance_x':'VeryActiveDistance','SedentaryActiveDistance_x':'SedentaryActiveDistance'},inplace = True)

In [ ]:
dailyStuff.head()

Now it looks good lol  

<a id="hourly"></a>  
##### Hourly data

In [ ]:
#Merging Hourly Data
hourlyStuff = pd.merge(hourlyCalories_merged,hourlyIntensities_merged,on = ["Id","ActivityHour"])
hourlyStuff = pd.merge(hourlyStuff,hourlySteps_merged,on = ["Id","ActivityHour"])

#creating a new column with hour of the day because it might be useful in the future
hourlyStuff['Hour'] = pd.to_datetime(hourlyStuff['ActivityHour']).dt.hour
hourlyStuff.head(4)

#### We can use SQL queries to join and format datasets as well. Below is how i created the dailyStuff dataset.
###### SQL queries to join the Daily Stuff
**select** * **from** fitbit.dailyActivity **as** da  
**FULL JOIN** fitbit.dailyCalories **as** dc  
**on** da.Id = dc.Id and da.ActivityDate  = dc.ActivityDay  
**FULL JOIN** fitbit.dailyIntensities as di  
**on** da.Id = di.id and dc.ActivityDay = di.ActivityDay  
**FULL JOIN** fitbit.dailySteps **as** ds  
**on** da.Id = ds.id and dc.ActivityDay = ds.ActivityDay  
**ORDER BY** da.ActivityDate   

But, I used Pandas to do the same thing because it integrates more and easier in this scenario.

Let's find out more about our datasets using info() function. This will give us information about data types of each column and we can adjust them if we need to

In [ ]:
dailyStuff.info()
dailyStuff_with_Sleep.info()

We can see that most data types assigned are correct. We will have to convert ActivityDay to a datetime type as well. And the other thing that concerns little bit is the User Id data type which is set to numeric. Since we used data collected from 33 differant users, we can set them to categorical type as well. But, Since we are not analyzing individuals' activities we can leave it as it is or convert to string type. We'll do the latter.

In [ ]:
dailyStuff['ActivityDay'] = pd.to_datetime(dailyStuff['ActivityDay'])
dailyStuff['Day'] = dailyStuff['ActivityDay'].dt.day_name()
dailyStuff_with_Sleep['ActivityDay'] = pd.to_datetime(dailyStuff_with_Sleep['ActivityDay'])
dailyStuff_with_Sleep['Day'] = dailyStuff_with_Sleep['ActivityDay'].dt.day_name()
dailyStuff['Id'] = dailyStuff['Id'].astype('string')
dailyStuff_with_Sleep['Id'] = dailyStuff_with_Sleep['Id'].astype('string')
#now its perfect

In [ ]:
dailyStuff.head()

In [ ]:
#Let's see about hourly Data
hourlyStuff.info()

We can see that Activity hour, Id, Hour are in the wrong format. I can convert it to string type. I will need to convert Activity Hour to datetime. Even though technically Hour column is also datetime, it's more useful in the categorical format for us. You'll see why later.

In [ ]:

hourlyStuff['Hour'] = hourlyStuff['Hour'].astype('category') #categorical
hourlyStuff['Id'] = hourlyStuff['Id'].astype('string')
#now its perfect

In [ ]:
hourlyStuff.head(4)

In [ ]:
#Checking  how many Participants are in our datasets
print(dailyStuff["Id"].nunique())
print(dailyStuff_with_Sleep["Id"].nunique())
print(hourlyStuff["Id"].nunique())

We can see that Dailystuff and hourlystuff data sets have 33 participants,  and daily with sleep have 24 participants

In [ ]:
#Checking if there are any missing values in our dataframes
print(dailyStuff_with_Sleep.isnull().values.any()) #gives us true or false, df.isnull().values.sum() = gives us number of missing values
print(dailyStuff.isnull().values.any())
print(hourlyStuff.isnull().values.any())

##### Hooray! we don't have any missing values! We can proceed with the analysis!
<a id="EDA"></a>  
## Exploratory Data Analysis


In [ ]:
from  matplotlib import pyplot as plt
import seaborn as sns

<a id="daily1"></a>  
#### Daily Data

In [ ]:
#Looking at daily average statistics
dailyStuff[['TotalSteps','TotalDistance','Calories','SedentaryMinutes']].mean()

Total steps amount is slightly less than the amount recommended by [CDC](https://www.cdc.gov/diabetes/prevention/pdf/postcurriculum_session8.pdf) which is 8000 Steps. According to [Healthline](https://www.healthline.com/nutrition/how-many-calories-per-day), Average caloric expenditure of females is 2100 calories and for males its 2600. In this dataset, gender of users is not available. So, mean value of 2300 is acceptable

In [ ]:
#Average Sleep Statistics
print(dailyStuff_with_Sleep[['TotalMinutesAsleep','TotalTimeInBed']].mean())
print("Ratio Between Average Sleep time and Average Total time in bed: "+
     str(dailyStuff_with_Sleep['TotalMinutesAsleep'].mean()/dailyStuff_with_Sleep['TotalTimeInBed'].mean()))

We can see that in Average Users are Asleep for 91% of the total time in Bed

In [ ]:
#Let's visualize average values of Types of Active Minutes
dailyActivityMean = dailyStuff.agg(
{'SedentaryMinutes': ['mean'],'LightlyActiveMinutes': ['mean'],
 'FairlyActiveMinutes': ['mean'],'VeryActiveMinutes': ['mean']}).reset_index()
plt.figure(figsize = (15,5))
plt1 = sns.barplot(data = dailyActivityMean)
plt1.set_xticklabels(plt1.get_xticklabels(), rotation=45)
plt1.set(xlabel='Level of Activeness', ylabel='Minutes',title = 'Average Activity Comparison')
dailyActivityMean

In [ ]:
#Let's visualize average distances travelled with Active Levels
#Sedentary Active Distance dont make any sense because you can you cant walk while being sedentary
#So, We'll drop it
dailyStuff.drop(['SedentaryActiveDistance'],inplace = True,axis = 1)
dailyStuff_with_Sleep.drop(['SedentaryActiveDistance'],inplace = True,axis = 1)
dailyDistanceMean = dailyStuff.agg(
{'LightActiveDistance': ['mean'],
 'ModeratelyActiveDistance': ['mean'],'VeryActiveDistance': ['mean']}).reset_index()
plt.figure(figsize = (15,5))
plt1 = sns.barplot(data = dailyDistanceMean)
plt1.set_xticklabels(plt1.get_xticklabels(), rotation=45)
plt1.set(xlabel='Level of Activeness', ylabel='Minutes',title = 'Average Travelled Distance Comparison')
dailyActivityMean

We can see that users are mostly lightly active when travelling.

In [ ]:
#Lets visualize How long users are active (Minutes) according to the day of the week.
#Getting total Active minutes (we don't need sedenatary because it's not really active)
dailyStuff['TotalActiveMinutes'] = dailyStuff[['LightlyActiveMinutes',
 'FairlyActiveMinutes','VeryActiveMinutes']].sum(axis = 1)
TotalActivitybyDay =  dailyStuff.groupby(['Day']).agg({'TotalActiveMinutes':'mean'}).reset_index()
df_mapping = pd.DataFrame({
    'Day': ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday','Saturday','Sunday']})
sort_mapping = df_mapping.reset_index().set_index('Day')
TotalActivitybyDay['DayNum'] = TotalActivitybyDay['Day'].map(sort_mapping['index'])
TotalActivitybyDay.sort_values(['DayNum'],inplace =True)
plt.figure(figsize = (15,5))
plt1 = sns.barplot(data = TotalActivitybyDay,x = "Day",y = "TotalActiveMinutes")
plt1.set_xticklabels(plt1.get_xticklabels(), rotation=45)
plt1.set(xlabel='Day of the Week', ylabel='Minutes',title = 'Average Total Active Minutes Vs Days of the week')

TotalActivitybyDay

We can see that there are no significant differences in active minutes by days of the week.

In [ ]:
#Comprehensive Days of the Week vs Active Sedentary Minutes
ActiveMinutesTypesbyDay = dailyStuff.groupby(['Day']).agg({
    'SedentaryMinutes':'mean','LightlyActiveMinutes':'mean',
    'FairlyActiveMinutes':'mean','VeryActiveMinutes':'mean'}).reset_index()
ActiveMinutesTypesbyDay['DayNum'] = ActiveMinutesTypesbyDay['Day'].map(sort_mapping['index'])
ActiveMinutesTypesbyDay.sort_values(['DayNum'],inplace =True)
ActiveMinutesTypesbyDay.drop('DayNum',inplace = True,axis = 1)

ActiveMinutesTypesbyDay


In order to plot correctly, We need to melt the dataset with respect to the Day column. This is sort of like converting Wide Table to Long Table

In [ ]:
melted_ActiveMinutesTypesbyDay = ActiveMinutesTypesbyDay.melt('Day',var_name = 'ActivityLevel',value_name = 'Minutes')
melted_ActiveMinutesTypesbyDay

In [ ]:
#Plotting
plt.figure(figsize = (15,5))
plt1 = sns.barplot(data = melted_ActiveMinutesTypesbyDay,x = 'Day',y = 'Minutes',hue = 'ActivityLevel')
plt1.set_xticklabels(plt1.get_xticklabels(), rotation=45)
plt1.set(xlabel='Day of the Week', ylabel='Minutes',title = 'Activity Level Vs Days of the week')
plt.show()


We can see that users have a consistant Activity Type Durations throughout the week.  
  
**Now Let's Do the Same for Distance**

In [ ]:
#Comprehensive Days of the Week vs Active Sedentary Minutes
DistanceTravelledTypesbyDay = dailyStuff.groupby(['Day']).agg({
    'LightActiveDistance':'mean','ModeratelyActiveDistance':'mean',
    'VeryActiveDistance':'mean'}).reset_index()
DistanceTravelledTypesbyDay['DayNum'] = ActiveMinutesTypesbyDay['Day'].map(sort_mapping['index'])
DistanceTravelledTypesbyDay.sort_values(['DayNum'],inplace =True)
DistanceTravelledTypesbyDay.drop('DayNum',inplace = True,axis = 1)

DistanceTravelledTypesbyDay

In [ ]:
#Melting the dataset
melted_DistanceTravelledTypesbyDay = DistanceTravelledTypesbyDay.melt('Day',var_name = 'DistanceType',value_name = 'Distance')
melted_DistanceTravelledTypesbyDay

In [ ]:
#Plotting the dataset
plt.figure(figsize = (15,5))
plt1 = sns.barplot(data = melted_DistanceTravelledTypesbyDay,x = 'Day',y = 'Distance',hue = 'DistanceType')
plt1.set_xticklabels(plt1.get_xticklabels(), rotation=45)
plt1.set(xlabel='Day of the Week', ylabel='Distance',title = 'Distance Type Vs Days of the week')
plt.show()

Even for Distance, We can see that users have a consistant Distance Type Distances throughout the week.  
  
  **Now, Let's visualise the relationship between Distance Travelled and Total Active Minutes**

In [ ]:
#Lets also draw a best fitting line / curve through the plot using Linear Regression
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree =5)

lin_reg = LinearRegression()
x = dailyStuff['TotalDistance'].values.reshape(-1,1)
y =dailyStuff['TotalActiveMinutes'].values.reshape(-1,1)
X_poly = poly_reg.fit_transform(x)
lin_reg.fit(X_poly,dailyStuff['TotalActiveMinutes'])
pred = lin_reg.predict(X_poly)

x_grid = np.arange(min(x),max(x),0.01)

plt.figure(figsize = (7,7))
plt.scatter(data = dailyStuff,x = 'TotalDistance',y = 'TotalActiveMinutes',color = 'darkblue')
plt.plot(x_grid ,lin_reg.predict(poly_reg.fit_transform(x_grid.reshape(-1,1))), color = 'red')
plt.xlabel("Total Distance")
plt.ylabel("Total Active Minutes")
plt.title("Total Active Minutes vs Total Distance Travelled")
plt.show()

We can see that their is a correlation between Total Active Minutes and Total Distance Travelled. Since the Fitbit fitness tracker collects Location (**GPS**) data, We can explain the outliers with Stationary exercising such as running on a trademill, stationary bike.. etc.  And this is not exactly a linear relationship. Which is to be expected with  this kind of data.

  **Now, Let's visualise the relationship between Calories Burned and Total Active Minutes**

In [ ]:
#Lets also draw a best fitting line / curve through the plot using Linear Regression
poly_reg = PolynomialFeatures(degree =3)
x = dailyStuff['TotalActiveMinutes'].values.reshape(-1,1)
y =dailyStuff['Calories'].values.reshape(-1,1)
X_poly = poly_reg.fit_transform(x)
lin_reg = LinearRegression()

lin_reg.fit(X_poly,y)
pred = lin_reg.predict(X_poly)

x_grid = np.arange(min(x),max(x),0.01)

plt.figure(figsize = (7,7))
plt.scatter(data = dailyStuff,x = 'TotalActiveMinutes',y = 'Calories',color = 'green')
plt.plot(x_grid ,lin_reg.predict(poly_reg.fit_transform(x_grid.reshape(-1,1))), color = 'yellow')
plt.xlabel("Total Active Minutes")
plt.ylabel("Daily Calories Burned")
plt.title("Daily Calories Burned vs Total Active Minutes")
plt.show()

We can observe a correlation between Daily Calories Burned and Total active Minutes. Obviously, THere will be a minimum amount of calories burned even if there is no active minutes. There are some incorrect data (where daily calories burned are zero) in our dataset. That data are incorrect because there it is impossible to not burn calories throughout the day. This maybe due to the user not wearing the fitness band throughout the day.

**It is kind of obvious that there is a postive correlation between Total steps taken and Total Distance travelled** So, We won't be looking at that data

**Another interesting thing is the relationship between Total Calories burned and the Amount of sleep user received. Let's Look into that right now.**

In [ ]:
poly_reg = PolynomialFeatures(degree =1)
x = dailyStuff_with_Sleep['TotalMinutesAsleep'].values.reshape(-1,1)
y =dailyStuff_with_Sleep['Calories'].values.reshape(-1,1)
X_poly = poly_reg.fit_transform(x)
lin_reg = LinearRegression()

lin_reg.fit(X_poly,y)
pred = lin_reg.predict(X_poly)

x_grid = np.arange(min(x),max(x),0.01)

plt.figure(figsize = (7,7))
plt.scatter(data = dailyStuff_with_Sleep,x = 'TotalMinutesAsleep',y = 'Calories',
            color = 'olive')
plt.plot(x_grid ,lin_reg.predict(poly_reg.fit_transform(x_grid.reshape(-1,1))),
         color = 'darkorange')
plt.xlabel("Total Sleep Received (Minutes)")
plt.ylabel("Daily Calories Burned")
plt.title("Total Sleep Received (Minutes) vs Calories Burned")
plt.show()

Suprisingly, the is no  visible correlation between Amount of Sleep and Calories Burned. But, we can see that most users are getting 300-600 minutes (5-10 hours) of sleep daily.  
  
  **Let's see if users are going to be more likely to be sedentary if they don't get enough sleep or not**

In [ ]:
poly_reg = PolynomialFeatures(degree =2)
x = dailyStuff_with_Sleep['TotalMinutesAsleep'].values.reshape(-1,1)
y =dailyStuff_with_Sleep['SedentaryMinutes'].values.reshape(-1,1)
X_poly = poly_reg.fit_transform(x)
lin_reg = LinearRegression()

lin_reg.fit(X_poly,y)
pred = lin_reg.predict(X_poly)

x_grid = np.arange(min(x),max(x),0.01)

plt.figure(figsize = (7,7))
plt.scatter(data = dailyStuff_with_Sleep,x = 'TotalMinutesAsleep',
            y = 'SedentaryMinutes',color = 'blueviolet')
plt.plot(x_grid ,lin_reg.predict(poly_reg.fit_transform(x_grid.reshape(-1,1))), color = 'gold')
plt.xlabel("Total Sleep Received (Minutes)")
plt.ylabel("Sedentary Minutes")
plt.title("Total Sleep Received (Minutes) vs Sedentary Minutes")
plt.show()

We can see that there is a negative correlation between Amount of Sleep Received and Sedentary Minutes. Users are more likely to be Sedentary for a long time if they dont get enough sleep. 

<a id="hourly1"></a>  
#### Hourly Data

In [ ]:
#Looking at Hourly average statistics
hourlyStuff[['Calories','TotalIntensity','StepTotal']].mean()

**Let's findout the Average calories burned throughout the day by Hour**

In [ ]:
hourlyCalories = hourlyStuff.groupby(['Hour']).agg({'Calories':'mean'}).reset_index()

plt.figure(figsize = (10,3))
hC = sns.barplot(data = hourlyCalories,x = 'Hour',y = 'Calories')
hC.set(xlabel='Hour of the Day', ylabel='Calories',title = 'Hour of the Day Vs Calories Burned')
plt.show()
#hourlyCalories

We can see that most of the calories are burned during the day time.  
  
  **Now Let's See Amount of Steps taken (Average) throughout the day by Hour**

In [ ]:
hourlySteps = hourlyStuff.groupby(['Hour']).agg({'StepTotal':'mean'}).reset_index()

plt.figure(figsize = (10,3))
hC = sns.barplot(data = hourlySteps,x = 'Hour',y = 'StepTotal')
hC.set(xlabel='Hour of the Day', ylabel='Amount of Steps',
       title = 'Hour of the Day Vs Amount of Steps Taken')
plt.show()
#hourlySteps

Unsuprisingly, during the night time, amount of steps taken is really low. Most amount of steps are taken during the day.  
  
  **Let's do the same thing for the Total Intensity**

In [ ]:
hourlyIntensity = hourlyStuff.groupby(['Hour']).agg({'TotalIntensity':'mean'}).reset_index()

plt.figure(figsize = (10,3))
hC = sns.barplot(data = hourlyIntensity,x = 'Hour',y = 'TotalIntensity')
hC.set(xlabel='Hour of the Day', ylabel='Hourly Intensity',
       title = 'Hour of the Day Vs Intensity')
plt.show()
#hourlyIntensity

<a id="predict"></a>  
## Predicting Daily Caloric with Polynomial Linear Regression

In [ ]:
#Choosing promising variables from the dailyStuff Dataset
dailyStuff_filtered = dailyStuff[['TotalSteps','TotalDistance','VeryActiveDistance','ModeratelyActiveDistance',
                                 'LightActiveDistance','VeryActiveMinutes','FairlyActiveMinutes','LightlyActiveMinutes',
                                 'SedentaryMinutes','TotalActiveMinutes','Calories']]
dailyStuff_filtered.head(4)

In [ ]:
# split into input (X) and output (Y) variables
X = dailyStuff_filtered.iloc[:,:-1] #first 10 variables 
y = dailyStuff_filtered.iloc[:,-1] #Calory Data

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)#20% test size


#Getting Polynomials from variables
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree =2)
X_poly = poly_reg.fit_transform(X_train)

#Fitting the data to LinearRegressionModel
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_poly,y_train)

#Prediction
y_pred = regressor.predict(poly_reg.fit_transform(X_test))

#Measuring Accuracy with Mean Squared Error
from sklearn.metrics import mean_squared_error 
print("Mean Squared Error is: " + str(mean_squared_error(y_test,y_pred)))

Then I optimized the model to improve prediction accuracy by removing some variables from the dataset.

In [ ]:

newdf = dailyStuff_filtered[['TotalSteps','TotalDistance','VeryActiveDistance',
                                 'LightActiveDistance','VeryActiveMinutes','LightlyActiveMinutes',
                                 'SedentaryMinutes','Calories']]
X = newdf.iloc[:,:-1]
y = newdf.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)#20% test size

poly_reg = PolynomialFeatures(degree =2)
X_poly = poly_reg.fit_transform(X_train)
regressor = LinearRegression()
regressor.fit(X_poly,y_train)

y_pred= regressor.predict(poly_reg.fit_transform(X_test))

mean_squared_error(y_test,y_pred)

print("NEW Mean Squared Error is: " + str(mean_squared_error(y_test,y_pred)))

<a id="insights"></a> 
### Insights  
* Clear positive correlation between activity and calories burned
* Less sleep causes users to be less productive. So improving
* Users are more likely to be active during the day time (duh...)
* Cannot find a relationship between sleep duration and amount of calories burned
* Longer a user is active, more likely for them to travel more (walk, run, etc..)
* Activity level, Amount of activity is don't change much with differant days of the week (slightly higher on saturdays)
* When travelling, users are most likely to be lightly active followed by very active, lastly moderately active  

<a id="recommend"></a> 
### Recommendations  
* If bellabeat users want to reduce sedentary time, recommend them to improve their sleeping (sleep longer)
* Recommend users who work sedentary jobs to exercise regularly
* Collect more data from the bellabeat users (with their permission) for better analysis
* Show bellabest users the benefits of increasing amount of steps taken. According to [CDC](https://www.cdc.gov/diabetes/prevention/pdf/postcurriculum_session8.pdf), more than 10000 steps per day reduces the risk of injuries, and reduces mortality in general.
* Suggest users recommended caloric intake based on their activity levels and their needs (lose weight, gain weight, etc.)  
  
  # Thank you!!!!!